# Finetune ruGPT3Small on your data


## Install enviroment

In [ ]:
!pip3 install urllib3==1.25.4

In [ ]:
!pip3 install transformers==2.8.0

In [ ]:
!wget https://raw.githubusercontent.com/sberbank-ai/ru-gpts/master/pretrain_transformers.py

In [ ]:
!wget https://raw.githubusercontent.com/sberbank-ai/ru-gpts/master/generate_transformers.py

In [ ]:
%%writefile setup.sh

export CUDA_HOME=/usr/local/cuda-10.1
git clone https://github.com/NVIDIA/apex
pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex

In [ ]:
!sh setup.sh

## Add data to colab


In [ ]:
from google.colab import drive 
drive.mount('content/')
data_path = './output.csv'

In [ ]:
import pandas as pd
data = pd.read_csv(data_path)
data.head()

In [ ]:
train = data.sample(frac=0.7,random_state=42).copy()
val = data[~data.index.isin(train.index)].copy()

In [ ]:
train = [train.text.iloc[idx] for idx in range(len(train)) if type(train.text.iloc[idx]) != float]
valid = [val.text.iloc[idx] for idx in range(len(val)) if type(val.text.iloc[idx]) != float]

In [ ]:
len(train), len(valid)

In [ ]:
with open("train.txt", "w") as file:
    file.write("\n".join(train))

In [ ]:
with open("valid.txt", "w") as file:
    file.write("\n".join(valid))

In [ ]:
!python pretrain_transformers.py \
    --output_dir=comment_model \
    --model_type=gpt2 \
    --model_name_or_path=sberbank-ai/rugpt3small_based_on_gpt2 \
    --do_train \
    --train_data_file=train.txt \
    --do_eval \
    --fp16 \
    --eval_data_file=valid.txt \
    --per_gpu_train_batch_size 1 \
    --gradient_accumulation_steps 1 \
    --num_train_epochs 5 \
    --block_size 2048 \
    --overwrite_output_dir

In [ ]:
!python generate_transformers.py \
    --model_type=gpt2 \
    --model_name_or_path=comment_model \
    --k=5 \
    --p=0.95 \
    --length=50